In [160]:
import spacy 
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.metrics import accuracy_score 
from sklearn.base import TransformerMixin 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
import string
punctuations = string.punctuation
!python3 -m spacy download en
spacy.load('en_core_web_sm')
from spacy.lang.en import English
parser = English()
import pandas as pd
import random
import inflect
p = inflect.engine()



✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [161]:
def datapreprocessing(url):

  # Load the dataset into a pandas dataframe.
  df = pd.read_csv(url, delimiter='\t', header=None, names=['sentence_source', 'label', 'label_notes', 'sentence'])

  sentences = df.sentence.values
  labels = df.label.values
  train=[]
  for i in range(len(df)):
    train.append((df.sentence[i],df.label[i]))
  
  random.shuffle(train)

  train_data = train[:7000]
  test_data = train[7000:]  
  return train_data,test_data

In [162]:
train_data,test_data=datapreprocessing('/in_domain_train.tsv')

In [163]:
def model():
  #Custom transformer using spaCy 
  class predictors(TransformerMixin):
      def transform(self, X, **transform_params):
          return [clean_text(text) for text in X]
      def fit(self, X, y=None, **fit_params):
          return self
      def get_params(self, deep=True):
          return {}

  # Basic utility function to clean the text 
  def clean_text(text):     
      return text.strip()
  def spacy_tokenizer(sentence):
      tokens = parser(sentence)
      tokens = [ tok for tok in tokens]
      return tokens

  #create vectorizer object to generate feature vectors, we will use custom spacy tokenizer
  vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1)) 
  classifier = LinearSVC()

  # Create the  pipeline to clean, tokenize, vectorize, and classify 
  pipe = Pipeline([("cleaner", predictors()),('vectorizer', vectorizer),('classifier', classifier)])
  
  return pipe

In [164]:
def detection(sen):
  pipe=model()
  pipe.fit([x[0] for x in train_data], [x[1] for x in train_data]) 
  pred_data = pipe.predict(sen) 
  result=pred_data[0]
  return result

In [165]:
def punct_data(sen):
  # Load the dataset into a pandas dataframe.
  df = pd.read_csv('/punct.csv')
  
  sentences = df.sentence.values
  labels = df.lables.values
  train=[]
  for i in range(len(df)):
    train.append((df.sentence[i],df.lables[i]))
  
  random.shuffle(train)

  train_data = train[:200]
  test_data = train[200:]  
  class predictors(TransformerMixin):
      def transform(self, X, **transform_params):
          return [clean_text(text) for text in X]
      def fit(self, X, y=None, **fit_params):
          return self
      def get_params(self, deep=True):
          return {}
  def clean_text(text):     
      return text.strip().lower()
  def spacy_tokenizer(sentence):
      tokens = parser(sentence)
      tokens = [ tok.lower_ for tok in tokens]
      return tokens

  #create vectorizer object to generate feature vectors, we will use custom spacy tokenizer
  vectorizer = CountVectorizer(tokenizer = spacy_tokenizer, ngram_range=(1,1)) 
  classifier = LinearSVC()

  # Create the  pipeline to clean, tokenize, vectorize, and classify 
  pipe = Pipeline([("cleaner", predictors()),('vectorizer', vectorizer),('classifier', classifier)])
  pipe.fit([x[0] for x in train_data], [x[1] for x in train_data]) 
  pred_data = pipe.predict(sen) 
  result=pred_data[0]
  print(result)
  return result

In [166]:
import spacy.cli
spacy.cli.download("en_core_web_lg")

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [167]:
def article(sen):
  import nltk
  import en_core_web_lg
  nlp = en_core_web_lg.load()
  doc = nlp(sen)
  art_word=[]
  art=[]
  fl=0
  article=['a','an']
  for i,word in enumerate(doc):
    if(fl==1):
      art_word.append(word)
      fl=0
    st=str(word)
    if st in article:
      art.append(st)
      fl=1
      pass
  for i,word in enumerate(art_word):
    st=str(word)
    q=p.a(st)
    if((art[i]+" "+st)==q):
      return 1
    else:
      return 0
  

In [169]:
def capital(sen):
  if sen[0].islower():
    print("Incorrect Capitalization")
  

In [170]:
sen=[]
sen.append(input("Enter Sentence"))
result=detection(sen[0])
if result==1:
  print("Sentence is Grammetically Correct")
else:
  print("Sentence is Grammetically Incorrect")
  punct=punctuation(sen[0])  
  if punct==0:
    print("Incorrect Punctuations")
  art=article(sen[0])  
  if art==0:
      print("Incorrect Article(a/an)")  
  capital(sen[0])   

Enter SentenceHello ,how are you?
Sentence is Grammetically Correct
